In [17]:
import random
import torch

# Generate synthetic data
data = []
for i in range(10000):
    age = random.randint(0, 100)
    age = torch.tensor(age, dtype=torch.float32)
    
    age = age / 100
    data.append((age, age))

# Save data to a file
with open("age_data.txt", "w") as f:
    for age, label in data:
        f.write(f"{age}\t{label}\n")


In [18]:
import torch.nn as nn


class AgeModel(nn.Module):
    def __init__(self):
        super(AgeModel, self).__init__()
        self.fc1 = nn.Linear(1, 10)
        self.fc2 = nn.Linear(10, 1)

    def forward(self, x):
        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    

def predict_age(model, age):
    with torch.no_grad():
        x = torch.tensor([age]).reshape(-1, 1)
        y_pred = model(x)
        return y_pred.item()


In [59]:
import torch.optim as optim

# Define hyperparameters
learning_rate = 0.01
batch_size = 64
num_epochs = 20

# Load data
data = []
with open("age_data.txt", "r") as f:
    for line in f:
        age, label = line.strip().split("\t")
        data.append((float(age), float(label)))

# Split data into training and validation sets
split_idx = int(len(data) * 0.8)
train_data = data[:split_idx]
val_data = data[split_idx:]

# Define model, loss function, and optimizer
model = AgeModel()
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

# Train the model
for epoch in range(num_epochs):
    # Shuffle training data
    random.shuffle(train_data)

    # Train on batches
    for i in range(0, len(train_data), batch_size):
        batch = train_data[i:i+batch_size]
        x = torch.tensor([d[0] for d in batch]).reshape(-1, 1)
        y = torch.tensor([d[1] for d in batch]).reshape(-1, 1)

        optimizer.zero_grad()
        outputs = model(x)
        loss = criterion(outputs, y)
        loss.backward()
        optimizer.step()

    # Compute validation loss
    val_loss = 0
    with torch.no_grad():
        for i in range(0, len(val_data), batch_size):
            batch = val_data[i:i+batch_size]
            x = torch.tensor([d[0] for d in batch]).reshape(-1, 1)
            y = torch.tensor([d[1] for d in batch]).reshape(-1, 1)

            outputs = model(x)
            val_loss += criterion(outputs, y).item()

    # Print training and validation loss for the current epoch
    print(
        f"Epoch {epoch+1}: Train Loss = {loss.item():.4f}, Val Loss = {val_loss/len(val_data):.4f}")


Epoch 1: Train Loss = 0.0318, Val Loss = 0.0006
Epoch 2: Train Loss = 0.0322, Val Loss = 0.0004
Epoch 3: Train Loss = 0.0175, Val Loss = 0.0003
Epoch 4: Train Loss = 0.0137, Val Loss = 0.0002
Epoch 5: Train Loss = 0.0115, Val Loss = 0.0002
Epoch 6: Train Loss = 0.0054, Val Loss = 0.0001
Epoch 7: Train Loss = 0.0052, Val Loss = 0.0001
Epoch 8: Train Loss = 0.0030, Val Loss = 0.0000
Epoch 9: Train Loss = 0.0018, Val Loss = 0.0000
Epoch 10: Train Loss = 0.0012, Val Loss = 0.0000
Epoch 11: Train Loss = 0.0005, Val Loss = 0.0000
Epoch 12: Train Loss = 0.0003, Val Loss = 0.0000
Epoch 13: Train Loss = 0.0002, Val Loss = 0.0000
Epoch 14: Train Loss = 0.0001, Val Loss = 0.0000
Epoch 15: Train Loss = 0.0001, Val Loss = 0.0000
Epoch 16: Train Loss = 0.0000, Val Loss = 0.0000
Epoch 17: Train Loss = 0.0000, Val Loss = 0.0000
Epoch 18: Train Loss = 0.0000, Val Loss = 0.0000
Epoch 19: Train Loss = 0.0000, Val Loss = 0.0000
Epoch 20: Train Loss = 0.0000, Val Loss = 0.0000


In [61]:
# Predict age based on user input
age = float(input("Enter your age: "))
age = age / 100
predicted_age = predict_age(model, age)
predicted_age = predicted_age * 100
print(f"Predicted age: {predicted_age:.0f}")


Predicted age: 26
